In [1]:
!mkdir -p ~/.cloudvolume/secrets
!cp /notebooks/*.json ~/.cloudvolume/secrets/
!pip install cloud-volume
!pip install scikit-image

    100% |################################| 71kB 2.2MB/s ta 0:00:011
    100% |################################| 81kB 3.5MB/s ta 0:00:011
    100% |################################| 419kB 2.3MB/s ta 0:00:01
    100% |################################| 92kB 4.5MB/s ta 0:00:011
    100% |################################| 71kB 5.4MB/s ta 0:00:011
    100% |################################| 133kB 4.2MB/s ta 0:00:01
    100% |################################| 92kB 5.4MB/s ta 0:00:011
    100% |################################| 194kB 3.6MB/s ta 0:00:01
    100% |################################| 7.1MB 212kB/s eta 0:00:01
    100% |################################| 51kB 12.1MB/s ta 0:00:01
    100% |################################| 143kB 9.8MB/s eta 0:00:01
    100% |################################| 143kB 9.8MB/s eta 0:00:01
    100% |################################| 266kB 5.5MB/s eta 0:00:01
    100% |################################| 61kB 11.7MB/s ta 0:00:01
    100% |####################

    100% |################################| 51kB 13.3MB/s ta 0:00:01
    100% |################################| 153kB 8.8MB/s eta 0:00:01
    100% |################################| 92kB 12.6MB/s ta 0:00:01
    100% |################################| 71kB 13.0MB/s ta 0:00:01
    100% |################################| 61kB 12.4MB/s ta 0:00:01
    100% |################################| 61kB 12.7MB/s ta 0:00:01
    100% |################################| 143kB 10.1MB/s ta 0:00:01
    100% |################################| 51kB 13.3MB/s ta 0:00:01
    100% |################################| 71kB 12.2MB/s ta 0:00:01
    100% |################################| 92kB 12.4MB/s ta 0:00:01
    100% |################################| 81kB 13.3MB/s ta 0:00:01
    100% |################################| 61kB 11.5MB/s ta 0:00:01
    100% |################################| 51kB 12.4MB/s ta 0:00:01
    100% |################################| 51kB 11.2MB/s ta 0:00:01
    100% |######################

    100% |################################| 51kB 12.2MB/s ta 0:00:01
    100% |################################| 92kB 12.5MB/s ta 0:00:01
    100% |################################| 81kB 12.6MB/s ta 0:00:01
    100% |################################| 552kB 2.9MB/s eta 0:00:01
    100% |################################| 51kB 11.8MB/s ta 0:00:01
    100% |################################| 102kB 12.0MB/s a 0:00:01
    100% |################################| 204kB 6.7MB/s eta 0:00:01
    100% |################################| 81kB 12.4MB/s ta 0:00:01
    100% |################################| 71kB 12.6MB/s ta 0:00:01
    100% |################################| 829kB 1.9MB/s eta 0:00:01
    100% |################################| 9.0MB 171kB/s eta 0:00:01
    100% |################################| 163kB 8.8MB/s eta 0:00:01


    100% |################################| 225kB 6.7MB/s eta 0:00:01
  Running setup.py bdist_wheel for posix-ipc ... done
  Stored in directory: /root/.cache/pip/wheels/5a/5c/17/d0e2d421abaf3b4097bf9db11108380d734195c6d15c24269d
  Running setup.py bdist_wheel for psutil ... done
  Stored in directory: /root/.cache/pip/wheels/fe/33/84/baea3d665de2d1af7e8f827f3883811bba5e4149443ccf8191
  Running setup.py bdist_wheel for tenacity ... done
  Stored in directory: /root/.cache/pip/wheels/cf/dd/89/99693d2e2c5954ccd7b7ba4fe9571b385712939d25e584a03a
  Running setup.py bdist_wheel for inflection ... done
  Stored in directory: /root/.cache/pip/wheels/9f/fa/4f/ef45a51543bd855e40494298824ad5e750109ad18fa80278ad
  Running setup.py bdist_wheel for Markdown ... done
  Stored in directory: /root/.cache/pip/wheels/34/2d/c2/75f4fd57c1b79e59f66388262ed37b69ceac77162988f8af63
  Running setup.py bdist_wheel for pandocfilters ... done
  Stored in directory: /root/.cache/pip/wheels/fc/a0/4c/ec26eaa3f225314

You are using pip version 9.0.3, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
    100% |################################| 25.3MB 56kB/s  eta 0:00:01
    100% |################################| 1.6MB 835kB/s eta 0:00:01
    100% |################################| 634kB 2.5MB/s eta 0:00:01
    100% |################################| 5.7MB 277kB/s eta 0:00:01
    100% |################################| 51kB 10.9MB/s ta 0:00:01
  Running setup.py bdist_wheel for networkx ... done
  Stored in directory: /root/.cache/pip/wheels/44/c0/34/6f98693a554301bdb405f8d65d95bbcd3e50180cbfdd98a94e
  Running setup.py bdist_wheel for toolz ... done
  Stored in directory: /root/.cache/pip/wheels/f4/0c/f6/ce6b2d1aa459ee97cc3c0f82236302bd62d89c86c700219463
Successfully built networkx toolz
You are using pip version 9.0.3, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [141]:
import os
import sys
import time
import torch
import torch.nn.functional as F
import torch.nn as nn
from torch.optim import lr_scheduler
from torch.autograd import Variable
import numpy as np
import collections
import random

from cloudvolume import CloudVolume
from cloudvolume.lib import Vec, Bbox
from skimage.transform import resize

def invert_mask(mask, th=50/255.0, bias=0.2):
        bitmask = mask < th
        th_mask = bitmask * mask
        th_max = np.max(th_mask) + bias
        return (th_max - th_mask) / th_max

class Stack():
    """Collection of CloudVolume paths, with load & save operations
    """

    def __init__(self, paths):
        self.paths = paths

    def load(self, key, bbox, src_mip=0, dst_mip=0):
        src_cv = CloudVolume(self.paths[key], mip=src_mip, fill_missing=True)
        src_bbox = Bbox.from_slices(src_cv.slices_from_global_coords(bbox))
        img = src_cv[src_bbox.to_slices()][:,:,0,0]
        if dst_mip != src_mip:
                dst_cv = CloudVolume(self.paths[key], mip=dst_mip, fill_missing=True)
                dst_bbox = Bbox.from_slices(dst_cv.slices_from_global_coords(bbox))
                img = resize(img, dst_bbox.size3()[:2], preserve_range=True)
        return img

    def save(self, key, mip, bbox, img, is_field=False):
        """Save vector field to CloudVolume

        Args:
        * cv: CloudVolume object
        * mip: int representing mip-level of the CloudVolume
        * bbox: Bbox at mip 0 with size3 of (x,y,1)
        * img: nxm int64 ndarray with shape (x,y)
        * is_field: boolean whether img is a field object
                field object is 4d tensor with shape (1,x,y,2)
        """
        src_cv = CloudVolume(self.paths['image'], mip=mip)
        info = src_cv.info
        if is_field:
                info['data_type'] = 'float32'
                info['num_channels'] = 2
        cv = CloudVolume(self.paths[key], mip=mip, bounded=False, 
                                         info=info, fill_missing=True, 
                                         non_aligned_writes=True, cdn_cache=False, progress=False)
        cv.commit_info()

        if is_field:
                s = cv.slices_from_global_coords(bbox) 
                s += (slice(0,2),)
                cv[s] = img.permute(1,2,0,3).data.cpu().numpy()
        else:
                s = cv.slices_from_global_coords(bbox)
                cv[s] = img[:,:,np.newaxis,np.newaxis]

class Optimizer():
    """

    Attributes:
        * src_image: nxm float64 ndarry normalized between [0,1]
            This is the image to be transformed by the returned vector field.
        * dst_images: list of nxm float64 ndarrays normalized between [0,1]
            This is the set of images that transformed src_image will be compared to.
        * src_smooth_mask: nxm float64 ndarray normalized between [0,1]
            The weight represents that degree to which a pixel participates in smooth
            deformation (0: not at all; 1: completely). This mask is used to reduce 
            the smoothness penalty for pixels that participate in a non-smooth 
            deformation.
            ### this mask is too severe on the smoothness penalty. consider changing
        * src_loss_mask: nxm float64 ndarray normalized between [0,1]
            This mask is transformed by the vector field and used to reduce the MSE
            for pixels that participate in a defect.
        * dst_masks_loss: list of nxm float64 ndarrays normalized between [0,1]
            Exactly like the src_mask_loss above. These masks are only used to reduce the 
            MSE for pixels that participate in a defect.
            ### unclear what `dst_masks_loss` is ???
    """

    def __init__(self, stack, bbox, dst_increment, image_mip, fold_mip, crack_mip, loss_mip, output_mip):
        self.stack = stack # the stack of images
        self.bbox = bbox # the "frame" in which to process
        self.image_mip = image_mip # lowest mip at which to process
        self.output_mip = output_mip # mip at which to produce the output image
        self.src_image = stack.load('image', bbox, src_mip=image_mip, dst_mip=image_mip) / 255.0
        self.src_fold = stack.load('fold', bbox, src_mip=fold_mip, dst_mip=image_mip) / 255.0
        self.src_crack = stack.load('crack', bbox, src_mip=crack_mip, dst_mip=image_mip) / 255.0
#         self.src_smooth_mask = invert_mask(np.minimum(self.src_fold, self.src_crack))
        self.src_smooth_mask = (256.0 - stack.load('widened', bbox, src_mip=fold_mip, dst_mip=image_mip)) / 255.0
        self.src_loss_mask = (stack.load('loss', bbox, src_mip=loss_mip, dst_mip=image_mip) == 0) * 1.0

        self.dst_images = []
        self.dst_masks = []
        for z in dst_increment: # run through the layers being compared/aligned to
            dst_bbox = self.bbox + [0,0,z]
            dst_image = stack.load('image', dst_bbox, src_mip=image_mip, dst_mip=image_mip)
            self.dst_images.append(dst_image)
            dst_mask = (stack.load('loss', dst_bbox, src_mip=loss_mip, dst_mip=image_mip) == 0) * 1.0
            self.dst_masks.append(dst_mask)
            if output_mip != image_mip:
                dst_image = stack.load('image', dst_bbox, src_mip=output_mip, dst_mip=output_mip)
            self.stack.save('output', output_mip, dst_bbox, dst_image, is_field=False)

        self.identities = {}
    
    # crops the image var by d/2 pixels on each edge of the dimensions specified by dims
    @staticmethod
    def center(var, dims, d):
        if not isinstance(d, collections.Sequence):
            d = [d for i in range(len(dims))]
        for idx, dim in enumerate(dims):
            if d[idx] == 0:
                continue
            var = var.narrow(dim, int(d[idx]/2), int(var.size()[dim] - d[idx]))
        return var
    
    # returns a grid containing the relative pixel location (unity under grid_sample())
    def get_identity_grid(self, dim, cache=True):
        if self is None or dim not in self.identities:
            gx, gy = np.linspace(-1, 1, dim), np.linspace(-1, 1, dim)
            I = np.stack(np.meshgrid(gx, gy))
            I = np.expand_dims(I, 0)
            I = torch.FloatTensor(I)
            I = torch.autograd.Variable(I, requires_grad=False)
            I = I.permute(0,2,3,1).cuda()
            if self is None:
                return I
            self.identities[dim] = I
        if cache:
            return self.identities[dim]
        else:
            return self.identities[dim].clone()

    def jacob(self, fields, symmetric = True):
        # field (batch, x loc, y loc, component)
        ### TODO include difference with zero at edges to encourage no change there
        def dx(f):
            p = Variable(torch.zeros((1,1,f.size(1),2))).cuda() # padding to maintain shape
            if symmetric:
                return torch.cat((p, f[:,2:,:,:] - f[:,:-2,:,:], p), 1)
            else:
                return torch.cat((f[:,1:,:,:] - f[:,:-1,:,:], p), 1)
        def dy(f):
            p = Variable(torch.zeros((1,f.size(1),1,2))).cuda()
            if symmetric:
                return torch.cat((p, f[:,:,2:,:] - f[:,:,:-2,:], p), 2)
            else:
                return torch.cat((f[:,:,1:,:] - f[:,:,:-1,:], p), 2)
        fields = sum(map(lambda f: [dx(f), dy(f)], fields), [])
        field = torch.sum(torch.cat(fields, -1) ** 2, -1)
        return field

    def penalty(self, fields, mask=1):
        jacob = self.jacob(fields)
#         mask = 1-(1-mask)*.95
        jacob = torch.mul(jacob, mask)
        return jacob
#         return torch.sum(jacob)

    def render(self, field):
        ### why 255? Is 1.0 a valid value then? Why divide it at all and not just convert to float?
        src = stack.load('image', bbox, src_mip=self.output_mip, dst_mip=self.output_mip) / 255.0
        src = torch.FloatTensor(src).cuda()
        src = Variable(src).unsqueeze(0).unsqueeze(0) # unsqueeze adds a dimension to the tensor
        ### a bit cluttered and redundant since check done twice and no need for lambda func. clean up
        if self.output_mip < self.image_mip: # upsample the field if needed
            upsample = lambda x: nn.Upsample(scale_factor=2**x, mode='bilinear') if x >0 else (lambda y: y)
            field = upsample(self.image_mip - self.output_mip)(field.permute(0,3,1,2)).permute(0,2,3,1)
        y = F.grid_sample(src, field + self.get_identity_grid(field.size(2)))
        img = y.data.cpu().numpy()
        img = (img[0,0,:,:]*255).astype(np.uint8)
        self.stack.save('output', self.output_mip, self.bbox, img, is_field=False)

    def process(self, field=None, crop=0, write_field=True, ndownsamples=4, 
                                                                     currn=5, avgn=20, lambda1=0.4, lr=0.2, 
                                                                     eps=0.01, min_iter=20, max_iter=1000,
                                                                     write_penalty = True):
        """Compute vector field that minimizes mean squared error (MSE) between 
        transformed src_image & all dst_images regularized by the smoothness of the 
        vector field subject to masks that allow the vector field to not be smooth. 
        The minimization uses SGD.

        Args:
        * field: nxmx2 float64 torch tensor normalized between [0,1]
            This field represents the derived vector field that transforms the 
            src_image subject to the contraints of the minimization.
        * ndownsamples: int
            The number of mip layers to use in the alignment process
        * lr: float
            The learning rate at the lowest mip level (adjusted at higher mips)
        * currn and avgn: int
            The number of current and prior samples of the cost function to compare for stoppage
        * crop: int
            crops the returned vector field by this many pixels on each side

        Returns:
        * field: nxmx2 float64 torch tensor normalized between [0,1]
        """
        downsample = lambda x: nn.AvgPool2d(2**x,2**x, count_include_pad=False) if x > 0 else (lambda y: y)
        upsample = nn.Upsample(scale_factor=2, mode='bilinear')
        
        # convert images to PyTorch Variables
        s = torch.FloatTensor(self.src_image)
        # note:unsqueeze adds a dimension to the tensor
        src_var = Variable((s - torch.mean(s)) / torch.std(s)).cuda().unsqueeze(0).unsqueeze(0)
        src_smooth_mask_var = Variable(torch.FloatTensor(self.src_smooth_mask)).cuda().unsqueeze(0).unsqueeze(0)
        src_loss_mask_var = Variable(torch.FloatTensor(self.src_loss_mask)).cuda().unsqueeze(0).unsqueeze(0)
        dst_vars = []
        dst_mask_vars = []
        for d, m in zip(self.dst_images, self.dst_masks):
            d = torch.FloatTensor(d)
            dst_var = Variable((d - torch.mean(d)) / torch.std(d)).cuda().unsqueeze(0).unsqueeze(0)
            dst_vars.append(dst_var)
            mask_var = Variable(torch.FloatTensor(m)).cuda().unsqueeze(0).unsqueeze(0)
            dst_mask_vars.append(mask_var)
        
        # initialize the field
        dim = int(src_var.size()[-1] / (2 ** (ndownsamples - 1)))
        if field is None:
            # to zeros
            field = Variable(torch.zeros((1,dim,dim,2))).cuda().detach()
#             # to random values
#             field = torch.rand((1,dim,dim,2))
#             f_mask = self.get_identity_grid(field.size(2)) * self.get_identity_grid(field.size(2))
# #             pad = 64/(2**(ndownsamples - 1))
# #             padding = (pad, pad, pad, pad)
# #             F.pad(self.center(field, (1,2), 128/(2**(ndownsamples - 1))),padding)
#             field = Variable(field).cuda().detach()
#             field = field * f_mask
        else:
            field = Variable(field).cuda().detach()
        field.requires_grad = True
        
        updates = 0
        for k in reversed(range(ndownsamples)): # mip levels (image_mip+ndownsamples-1) through image_mip
            src_ = downsample(k)(src_var).detach()
            src_.requires_grad = False    ### isn't this redundant? detach already sets this
            src_smooth_mask_ = downsample(k)(src_smooth_mask_var).detach()
            src_smooth_mask_.requires_grad = False
            src_loss_mask_ = downsample(k)(src_loss_mask_var).detach()
            src_loss_mask_.requires_grad = False
            dst_list_, dst_mask_list_ = [], []
            for d, m in zip(dst_vars, dst_mask_vars):
                dst_ = downsample(k)(d).detach()
                dst_.requires_grad = False
                dst_list_.append(dst_)
                mask_ = downsample(k)(m).detach()
                mask_.requires_grad = False
                dst_mask_list_.append(mask_)
            field = field.detach() ### why, if already detached above?
            field.requires_grad = True # not redundant (detach makes it False by default)
            opt = torch.optim.SGD([field], lr=lr/(k+1)) ### why is the learning rate increased for lower mips??
            #sched = lr_scheduler.StepLR(opt, step_size=1, gamma=0.995)
            costs = [] # list of decreasing costs over time
            start_updates = updates ### not used (?)
            print('Downsample factor {0}x'.format(2**k))
            while True:
                updates += 1
                I = self.get_identity_grid(field.size(2))
                pred = F.grid_sample(src_, field + I)
#                 print("image_mip + k:", image_mip + k)
                pred_loss_mask = F.grid_sample(src_loss_mask_, field + I)
                pred_smooth_mask = F.grid_sample(src_smooth_mask_, field + I)
                ### why is 128 hardcoded?
                # squeeze removes a dimension from the tensor
                centered_mask = self.center(pred_smooth_mask.squeeze(0), (1,2), 128/(2**k))
#                 centered_mask = self.center(pred_loss_mask.squeeze(0), (1,2), 128/(2**k))
                centered_field = self.center(field, (1,2), 128/(2**k))
                # calaculate the smoothness penalty
                ### currently the masks are transformed with the field, so if the field is wrong,
                ### so is the mask (!)
                penalty_error_image = self.penalty([centered_field], centered_mask)
                penalty1 = torch.sum(penalty_error_image)
                cost = penalty1 * lambda1/(k+1) ### why divide by (k+1) ?
                for d, m in zip(dst_list_, dst_mask_list_):
                    # merge the pixel-perfect mask from the neighboring slice with that of the current slice
                    mask = torch.mul(pred_loss_mask, m) # all 0.0 or 1.0:   0 := ignore
                    mse = torch.mul(pred - d, mask)**2 # mse with the neighboring slice
                    ### why -1,-2? what are the dimensions of mse?
                    mse_error_image = self.center(mse, (-1,-2), 128 / (2**k))
                    cost += torch.mean(mse_error_image)
#                     print ("penalty_error_image: ", penalty_error_image.size())
#                     print ("mse_error_image: ", mse_error_image.size())
#                     penalty_error_image += mse_error_image
                #cost = diff + penalty1 * lambda1/(k+1)
                print(cost.data.cpu().numpy())
                costs.append(cost)
                cost.backward()
                opt.step()
                #sched.step()uniform
                opt.zero_grad()
                if len(costs) > avgn + currn and len(costs)>min_iter:
                    ### can already break here if len(costs)>max_iter
                    hist_costs = costs[-(avgn+currn):-currn]
                    hist = sum(hist_costs).data[0] / avgn
                    curr = sum(costs[-currn:]).data[0] / currn
                    ### why does the epsilon increase for lower mip levels? Is that because of larger cost?
                    if abs((hist-curr)/(hist+.0000001)) < eps/(2**k) or len(costs)>max_iter:
                        ### may be useful to know which of these cases happens. maybe print
                        break
                #print (downsamples, updates - start_updates)
            if write_field:
                self.stack.save('field', self.image_mip+k, self.bbox, field + I, is_field=True)
            if write_penalty:
                # output the smoothness penalty
                print("before:", penalty_error_image.size())
                img = penalty_error_image.squeeze(0).data.cpu().numpy()
                img_max = np.max(img) # max value to rescale the image
                print("smoothness penalty img_max:", img_max)
                img = (img*255/img_max).astype(np.uint8)
                v = Vec(64*(2**image_mip),64*(2**image_mip),0)
                bbox = Bbox(self.bbox.minpt + v, self.bbox.maxpt - v)
                print ("self:", self.bbox)
                print ("modified:", bbox)
                self.stack.save('penalty', self.image_mip+k, bbox, img, is_field=False)
                # output the mse
                img = mse_error_image.squeeze(0).squeeze(0).data.cpu().numpy()
                img_max = np.max(img) # max value to rescale the image
                print("mse img_max:", img_max)
                img = (img*255/img_max).astype(np.uint8)
                v = Vec(64*(2**image_mip),64*(2**image_mip),0)
                bbox = Bbox(self.bbox.minpt + v, self.bbox.maxpt - v)
                self.stack.save('mse', self.image_mip+k, bbox, img, is_field=False)
            if k > 0:
                field = upsample(field.permute(0,3,1,2)).permute(0,2,3,1)
        #print(cost.data[0], diff.data[0], penalty1.data[0])
        print('done:', updates)
        print(field.shape)
        # return self.center(field, (1,2), crop*2).data.cpu().numpy()[0]
        return self.center(field, (1,2), crop*2)

In [144]:
name = 'optimizertests_grad'
paths = {
    'image': 'gs://neuroglancer/basil_v0/son_of_alignment/v3.04/unmasked',
    'fold': 'gs://neuroglancer/basil_v0/son_of_alignment/v3.04/mask/fold_detector_v1',
    'crack': 'gs://neuroglancer/basil_v0/son_of_alignment/v3.04/mask/crack_detector_v3',
    'loss': 'gs://neuroglancer/basil_v0/son_of_alignment/v3.04/mask/ground_truth',
    'output': 'gs://neuroglancer/basil_v0/son_of_alignment/v3.04/optimizer_tests_bn/{0}'.format(name),
    'field': 'gs://neuroglancer/basil_v0/son_of_alignment/v3.04/optimizer_tests_bn/{0}/field'.format(name),
    'penalty': 'gs://neuroglancer/basil_v0/son_of_alignment/v3.04/optimizer_tests_bn/{0}/penalty'.format(name),
    'mse': 'gs://neuroglancer/basil_v0/son_of_alignment/v3.04/optimizer_tests_bn/{0}/mse'.format(name),
    'widened': 'gs://neuroglancer/basil_v0/son_of_alignment/v3.04/optimizer_tests_bn/{0}/widened'.format(name),
}
stack = Stack(paths)
offsets = [
              Vec(102716, 107077, 526), # fold
              Vec(102320, 111106, 526), # crack
#               Vec(102565, 103904, 527), # no defect
              Vec(104799, 109774, 526) # test (remove later)
          ]
size = Vec(2048, 2048, 1)
image_mip = 2
fold_mip = 5
crack_mip = 5
loss_mip = 2
output_mip = 1
dst_increment = [-1,1]
for offset in offsets:
    bbox = Bbox(offset, offset+size)
    o = Optimizer(stack, bbox, dst_increment, image_mip, fold_mip, crack_mip, loss_mip, output_mip)
    field = o.process(ndownsamples=3, currn=5, avgn=20, lambda1=0.9, lr=0.01, 
                                                  eps=0.001, min_iter=100, max_iter=5000, 
                                                  field=field_init.data.cpu())
    o.render(field)

Downloading:   0%|          | 0/1 [00:00<?, ?it/s]
/usr/local/lib/python3.5/dist-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/usr/local/lib/python3.5/dist-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "
Downloading:   0%|          | 0/4 [00:00<?, ?it/s]


Downsample factor 4x
[1.0080129]
[0.99994457]
[0.9920534]
[0.98454654]
[0.97736484]
[0.97045374]
[0.9638184]
[0.9574165]
[0.95123607]
[0.94525445]
[0.93947625]
[0.93359715]
[0.9280399]
[0.9227324]
[0.917585]
[0.9126343]
[0.907846]
[0.9031937]
[0.8986553]
[0.8942333]
[0.8899231]
[0.88572526]
[0.881642]
[0.8776874]
[0.87373996]
[0.8699913]
[0.8663902]
[0.86290884]
[0.85955846]
[0.8563619]
[0.8532641]
[0.8502776]
[0.8473841]
[0.8446053]
[0.8418759]
[0.8392242]
[0.8366746]
[0.83418995]
[0.83178854]
[0.8294526]
[0.8271849]
[0.8249707]
[0.8228357]
[0.82074225]
[0.81874037]
[0.8168189]
[0.81495583]
[0.81316626]
[0.8114395]
[0.80973536]
[0.8080908]
[0.8064743]
[0.8048875]
[0.80335784]
[0.80185246]
[0.8003994]
[0.79897296]
[0.7975893]
[0.7962796]
[0.7949844]
[0.79371953]
[0.79249537]
[0.7913171]
[0.790105]
[0.7889753]
[0.78787637]
[0.786793]
[0.7857144]
[0.7846837]
[0.7836897]
[0.7826624]
[0.7816937]
[0.7807534]
[0.7798016]
[0.77887166]
[0.77796376]
[0.77707934]
[0.77620107]
[0.7753434]
[0.7745

[0.6861764]
[0.68611115]
[0.68609166]
[0.6860389]
[0.6859932]
[0.6859666]
[0.6859306]
[0.68587196]
[0.68582857]
[0.6858134]
[0.6857665]
[0.68569505]
[0.68568695]
[0.68565404]
[0.6855976]
[0.6855484]
[0.6855238]
[0.6854843]
[0.68544674]
[0.6854044]
[0.68537104]
[0.6853126]
[0.68528914]
[0.6852722]
[0.6852039]
[0.68516785]
[0.6851257]
[0.68510485]
[0.68506885]
[0.68502367]
[0.6849824]
[0.68493026]
[0.6849162]
[0.6848882]
[0.6848084]
[0.68477035]
[0.6847609]
[0.68472683]
[0.68467414]
[0.684639]
[0.6846049]
[0.68456423]
[0.6845336]
[0.6844814]
[0.68445385]
[0.6844233]
[0.68438506]
[0.6843479]
[0.68431437]
[0.6842799]
[0.684235]
[0.6842154]
[0.68417]
[0.684124]
[0.6841]
[0.6840575]
[0.6840223]
[0.6839907]
[0.68396324]
[0.6839023]
[0.6838616]
[0.68383]
[0.68380725]
[0.68378603]
[0.68371975]
[0.68367803]
[0.68365014]
[0.6836251]
[0.6835896]
[0.68353164]
[0.6835066]
[0.6834705]
[0.68344855]
[0.68339294]
[0.6833534]
[0.6833464]
[0.683296]
[0.68324757]
[0.68320954]
[0.6831826]
[0.68315923]
[0.68

[1.2200955]
[1.2187903]
[1.2175353]
[1.2163115]
[1.2151265]
[1.2139832]
[1.212864]
[1.2117784]
[1.2107226]
[1.2096748]
[1.2086499]
[1.2076747]
[1.206728]
[1.2058039]
[1.204891]
[1.2039828]
[1.2031188]
[1.2022735]
[1.2014629]
[1.200665]
[1.1998878]
[1.1991391]
[1.1983933]
[1.1976833]
[1.1969538]
[1.1962578]
[1.1955581]
[1.1948712]
[1.1942301]
[1.1935979]
[1.1929853]
[1.1923864]
[1.1917979]
[1.1912267]
[1.1906617]
[1.1901126]
[1.1895733]
[1.18906]
[1.1885307]
[1.188015]
[1.1875198]
[1.1870164]
[1.1865296]
[1.1860523]
[1.1855845]
[1.1851335]
[1.184687]
[1.1842525]
[1.183819]
[1.1834042]
[1.182987]
[1.1825781]
[1.1821864]
[1.1817932]
[1.1814188]
[1.1810352]
[1.1806726]
[1.180305]
[1.1799531]
[1.1796064]
[1.179258]
[1.1789275]
[1.1785843]
[1.1782653]
[1.1779368]
[1.177613]
[1.1772975]
[1.1769733]
[1.1766654]
[1.17636]
[1.1760535]
[1.1757478]
[1.1754563]
[1.1751745]
[1.1748881]
[1.174614]
[1.1743438]
[1.1740744]
[1.1738116]
[1.1735375]
[1.1732754]
[1.1730175]
[1.172767]
[1.1725136]
[1.172259

Downloading:   0%|          | 0/4 [00:00<?, ?it/s]


...

Downsample factor 4x
[1.1077894]
[1.0980246]
[1.0879965]
[1.078515]
[1.0695577]
[1.0610709]
[1.053019]
[1.0453722]
[1.0381204]
[1.031222]
[1.0245957]
[1.0183009]
[1.0122986]
[1.0065653]
[1.0010495]
[0.9957516]
[0.9907005]
[0.9858373]
[0.98118585]
[0.97670275]
[0.9724093]
[0.9682907]
[0.96433365]
[0.960538]
[0.9568925]
[0.9533948]
[0.95002383]
[0.9467854]
[0.94365054]
[0.9406426]
[0.93772984]
[0.93491054]
[0.932191]
[0.92957246]
[0.92703784]
[0.92457175]
[0.9221841]
[0.9198779]
[0.9176461]
[0.9154901]
[0.913404]
[0.9113878]
[0.9094256]
[0.9075198]
[0.90568084]
[0.90389067]
[0.90216166]
[0.90047634]
[0.89881194]
[0.8971964]
[0.89563036]
[0.8941166]
[0.892637]
[0.89120245]
[0.88980865]
[0.88844323]
[0.88712]
[0.88582516]
[0.8845678]
[0.883317]
[0.88209987]
[0.88091373]
[0.87976426]
[0.8786384]
[0.8775369]
[0.8764765]
[0.87542915]
[0.874399]
[0.87335473]
[0.87233603]
[0.87134826]
[0.8703753]
[0.8694334]
[0.8685038]
[0.8676045]
[0.86672235]
[0.8658578]
[0.86502165]
[0.86418676]
[0.8633511]


[0.7709652]
[0.7709178]
[0.7708708]
[0.77082765]
[0.7707818]
[0.7707381]
[0.7706934]
[0.77064633]
[0.7706028]
[0.770561]
[0.77051276]
[0.7704682]
[0.7704258]
[0.7703796]
[0.7703361]
[0.7702937]
[0.7702458]
[0.77020806]
[0.7701634]
[0.7701171]
[0.77007604]
[0.770033]
[0.76999223]
[0.76994854]
[0.76990604]
[0.7698612]
[0.769822]
[0.769777]
[0.76973486]
[0.7696893]
[0.7696475]
[0.7696023]
[0.7695595]
[0.76951885]
[0.76947117]
[0.7694327]
[0.76939154]
[0.76934683]
[0.76930046]
[0.7692665]
[0.769217]
[0.7691777]
[0.76913834]
[0.7690908]
[0.76905584]
[0.76901376]
[0.7689696]
[0.7689328]
[0.7688869]
[0.7688527]
[0.7688086]
[0.7687665]
[0.76872885]
[0.7686858]
[0.7686499]
[0.7686049]
[0.7685666]
[0.76852584]
[0.7684909]
[0.76844347]
[0.7684126]
[0.76836085]
[0.7683201]
[0.7682776]
[0.7682302]
[0.7681904]
[0.7681456]
[0.7681043]
[0.7680601]
[0.76802087]
[0.76797295]
[0.7679362]
[0.76789284]
[0.76785225]
[0.7678094]
[0.76776916]
[0.7677299]
[0.7676851]
[0.76764685]
[0.7676053]
[0.76756155]
[0.76

[0.7501344]
[0.75011027]
[0.7500813]
[0.7500557]
[0.7500298]
[0.74999857]
[0.74997854]
[0.7499482]
[0.74992645]
[0.7498964]
[0.749872]
[0.74985194]
[0.7498243]
[0.7498015]
[0.74977756]
[0.7497499]
[0.749733]
[0.749704]
[0.74968374]
[0.7496588]
[0.7496358]
[0.74961233]
[0.74958414]
[0.7495604]
[0.7495321]
[0.7495073]
[0.74948215]
[0.7494569]
[0.7494288]
[0.7494105]
[0.7493818]
[0.7493601]
[0.7493344]
[0.7493084]
[0.7492838]
[0.74926424]
[0.74923337]
[0.7492156]
[0.74918544]
[0.7491646]
[0.74914324]
[0.74911726]
[0.7490938]
[0.7490728]
[0.74904704]
[0.74902403]
[0.7490026]
[0.74897695]
[0.748955]
[0.7489345]
[0.74890935]
[0.748886]
[0.748863]
[0.7488427]
[0.7488192]
[0.7487991]
[0.74877656]
[0.7487529]
[0.7487366]
[0.7487122]
[0.74869174]
[0.74866974]
[0.74864876]
[0.7486301]
[0.74860984]
[0.74858403]
[0.7485702]
[0.7485446]
[0.74852777]
[0.74848807]
[0.74845195]
[0.74841356]
[0.74837977]
[0.748345]
[0.74831355]
[0.7482744]
[0.7482455]
[0.74821484]
[0.74818027]
[0.74815273]
[0.7481211]
[

[1.2739465]
[1.2736661]
[1.2733932]
[1.2731174]
[1.2728508]
[1.272577]
[1.2723072]
[1.2720479]
[1.2717569]
[1.2714734]
[1.2711928]
[1.2709241]
[1.2706609]
[1.2704017]
[1.2701492]
[1.2699062]
[1.2696569]
[1.2694222]
[1.2691853]
[1.2689555]
[1.2687232]
[1.2684984]
[1.2682768]
[1.2680568]
[1.2678387]
[1.2676284]
[1.2674174]
[1.2672133]
[1.2670054]
[1.2668003]
[1.2665999]
[1.2663879]
[1.2661767]
[1.2659688]
[1.2657592]
[1.2655519]
[1.2653545]
[1.2651521]
[1.2649529]
[1.2647626]
[1.26457]
[1.264384]
[1.2641987]
[1.2640148]
[1.2638326]
[1.2636466]
[1.2634692]
[1.2632883]
[1.2631142]
[1.2629446]
[1.2627722]
[1.2626064]
[1.2624424]
[1.262279]
[1.2621188]
[1.26196]
[1.2617974]
[1.2616438]
[1.2614913]
[1.2613345]
[1.2611859]
[1.2610371]
[1.260891]
[1.2607402]
[1.2605976]
[1.260452]
[1.2603083]
[1.2601674]
[1.2600269]
[1.259885]
[1.2597492]
[1.2596046]
[1.2594714]
[1.2593353]
[1.2592]
[1.2590663]
[1.2589304]
[1.2588005]
[1.2586701]
[1.2585356]
[1.2584083]
[1.2582763]
[1.2581494]
[1.2580229]
[1.25

Downloading:   0%|          | 0/4 [00:00<?, ?it/s]


In [2]:
from scipy import ndimage
kernel_size = 101
# kernel = np.array([[np.sqrt(np.sqrt(np.sqrt(np.sqrt(np.sqrt(np.sqrt((i-kernel_size//2)**2 + (j-kernel_size//2)**2)))))) for j in range(kernel_size)] for i in range(kernel_size)])
kernel = np.array([[np.sqrt((i-kernel_size//2)**2 + (j-kernel_size//2)**2) for j in range(kernel_size)] for i in range(kernel_size)])
kernel = 255 - kernel/np.max(kernel)*255
[[kernel[i,j] for i in range(0,kernel_size,kernel_size//4)] for j in range(0,kernel_size,kernel_size//4)]

[[0.0, 53.404799164265825, 74.6877707974304, 53.404799164265825, 0.0],
 [53.404799164265825, 127.5, 164.8438853987152, 127.5, 53.404799164265825],
 [74.6877707974304,
  164.8438853987152,
  255.0,
  164.8438853987152,
  74.6877707974304],
 [53.404799164265825, 127.5, 164.8438853987152, 127.5, 53.404799164265825],
 [0.0, 53.404799164265825, 74.6877707974304, 53.404799164265825, 0.0]]

In [30]:
# widen pixel-perfect mask
name = 'optimizertests_grad'
paths = {
    'image': 'gs://neuroglancer/basil_v0/son_of_alignment/v3.04/unmasked',
    'fold': 'gs://neuroglancer/basil_v0/son_of_alignment/v3.04/mask/fold_detector_v1',
    'crack': 'gs://neuroglancer/basil_v0/son_of_alignment/v3.04/mask/crack_detector_v3',
    'loss': 'gs://neuroglancer/basil_v0/son_of_alignment/v3.04/mask/ground_truth',
    'output': 'gs://neuroglancer/basil_v0/son_of_alignment/v3.04/optimizer_tests_bn/{0}'.format(name),
    'field': 'gs://neuroglancer/basil_v0/son_of_alignment/v3.04/optimizer_tests_bn/{0}/field'.format(name),
    'penalty': 'gs://neuroglancer/basil_v0/son_of_alignment/v3.04/optimizer_tests_bn/{0}/penalty'.format(name),
    'mse': 'gs://neuroglancer/basil_v0/son_of_alignment/v3.04/optimizer_tests_bn/{0}/mse'.format(name),
    'widened': 'gs://neuroglancer/basil_v0/son_of_alignment/v3.04/optimizer_tests_bn/{0}/widened'.format(name),
}
stack = Stack(paths)
offsets = [
              Vec(102716, 107077, 526), # fold
              Vec(102320, 111106, 526), # crack
#               Vec(102565, 103904, 527), # no defect
              Vec(104799, 109774, 526) # test (remove later)
          ]
size = Vec(2048, 2048, 1)
image_mip = 2
fold_mip = 5
crack_mip = 5
loss_mip = 2
output_mip = 1
dst_increment = [-1,1]

for offset in offsets:
    bbox = Bbox(offset, offset+size)
    pixelperfect_mask = stack.load('loss', bbox, src_mip=loss_mip, dst_mip=image_mip).astype(np.float64)

    widened = ndimage.filters.convolve(pixelperfect_mask, kernel, mode='constant', cval=0.0)
    widened = (widened / (np.max(widened)+0.0) * 255).astype(np.uint8)
    stack.save('widened', loss_mip, bbox, widened, is_field=False)

Downloading:   0%|          | 0/4 [00:00<?, ?it/s]
Rechunking image: 0it [00:00, ?it/s]
Uploading: 0it [00:00, ?it/s]
Rechunking image:   0%|          | 0/1 [00:00<?, ?it/s]

Rechunking image:   0%|          | 0/1 [00:00<?, ?it/s]


Rechunking image:   0%|          | 0/1 [00:00<?, ?it/s]



Rechunking image:   0%|          | 0/1 [00:00<?, ?it/s]

Rechunking image: 100%|██████████| 1/1 [00:00<00:00,  6.59it/s]

Downloading:   0%|          | 0/4 [00:00<?, ?it/s]
Rechunking image: 0it [00:00, ?it/s]
Uploading: 0it [00:00, ?it/s]
Rechunking image:   0%|          | 0/1 [00:00<?, ?it/s]

Rechunking image:   0%|          | 0/1 [00:00<?, ?it/s]


Rechunking image:   0%|          | 0/1 [00:00<?, ?it/s]



Downloading:   0%|          | 0/2 [00:00<?, ?it/s]
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:34: RuntimeWarning: invalid value encountered in true_divide
Rechunking image: 0it [00:00, ?it/s]
Uploading: 0it [00:00, ?it/s]
Rechunking image:   0%|          | 0/1 [00:00<?, ?it/s

In [29]:
from scipy import ndimage
kernel_size = 101
# kernel = np.array([[np.sqrt(np.sqrt(np.sqrt(np.sqrt(np.sqrt(np.sqrt((i-kernel_size//2)**2 + (j-kernel_size//2)**2)))))) for j in range(kernel_size)] for i in range(kernel_size)])
kernel = np.array([[np.sqrt((i-kernel_size//2)**2 + (j-kernel_size//2)**2) for j in range(kernel_size)] for i in range(kernel_size)])
kernel = 255 - kernel/np.max(kernel)*255
[[kernel[i,j] for i in range(0,kernel_size,kernel_size//4)] for j in range(0,kernel_size,kernel_size//4)]

[[0.0, 53.404799164265825, 74.6877707974304, 53.404799164265825, 0.0],
 [53.404799164265825, 127.5, 164.8438853987152, 127.5, 53.404799164265825],
 [74.6877707974304,
  164.8438853987152,
  255.0,
  164.8438853987152,
  74.6877707974304],
 [53.404799164265825, 127.5, 164.8438853987152, 127.5, 53.404799164265825],
 [0.0, 53.404799164265825, 74.6877707974304, 53.404799164265825, 0.0]]

In [145]:
from skimage.filters import gaussian

# initialize vector field to gradient of "widened"
name = 'optimizertests_grad'
paths = {
    'image': 'gs://neuroglancer/basil_v0/son_of_alignment/v3.04/unmasked',
    'fold': 'gs://neuroglancer/basil_v0/son_of_alignment/v3.04/mask/fold_detector_v1',
    'crack': 'gs://neuroglancer/basil_v0/son_of_alignment/v3.04/mask/crack_detector_v3',
    'loss': 'gs://neuroglancer/basil_v0/son_of_alignment/v3.04/mask/ground_truth',
    'output': 'gs://neuroglancer/basil_v0/son_of_alignment/v3.04/optimizer_tests_bn/{0}'.format(name),
    'field': 'gs://neuroglancer/basil_v0/son_of_alignment/v3.04/optimizer_tests_bn/{0}/field'.format(name),
    'penalty': 'gs://neuroglancer/basil_v0/son_of_alignment/v3.04/optimizer_tests_bn/{0}/penalty'.format(name),
    'mse': 'gs://neuroglancer/basil_v0/son_of_alignment/v3.04/optimizer_tests_bn/{0}/mse'.format(name),
    'widened': 'gs://neuroglancer/basil_v0/son_of_alignment/v3.04/optimizer_tests_bn/{0}/widened'.format(name),
    'expand': 'gs://neuroglancer/basil_v0/son_of_alignment/v3.04/optimizer_tests_bn/{0}/expand'.format(name),
}
stack = Stack(paths)
offsets = [
              Vec(102716, 107077, 526), # fold
              Vec(102320, 111106, 526), # crack
#               Vec(102565, 103904, 527), # no defect
              Vec(104799, 109774, 526) # test (remove later)
          ]
size = Vec(2048, 2048, 1)
image_mip = 2
fold_mip = 5
crack_mip = 5
loss_mip = 2
output_mip = 1
dst_increment = [-1,1]

already = False

for offset in offsets:
    bbox = Bbox(offset, offset+size)
    
    pixelperfect_mask = stack.load('loss', bbox, src_mip=loss_mip, dst_mip=0).astype(np.float64)

    widened = gaussian(pixelperfect_mask, sigma=120)
    widened = (widened / (np.max(widened)+0.0) * 255)
    stack.save('widened', 0, bbox, widened.astype(np.uint8), is_field=False)
    
    image = stack.load('image', bbox, src_mip=image_mip, dst_mip=image_mip).astype(np.float64) / 255.0
#     widened = stack.load('widened', bbox, src_mip=loss_mip, dst_mip=loss_mip).astype(np.float64)
    
    field = np.gradient(widened)
    temp = field[0]
    field[0] = field[1]
    field[1] = temp
    field = field/np.max(field)/50*1.8
    field1 = field
    print(field1, np.max(field1))
    print("field shape:", np.shape(field))
    print("image shape:", np.shape(image))
    
    # convert images to PyTorch Variables
    s = torch.FloatTensor(image)
    # note:unsqueeze adds a dimension to the tensor
#     s = (s - torch.mean(s)) / torch.std(s)
    image = Variable(s).cuda().unsqueeze(0).unsqueeze(0)
    
    field = np.expand_dims(field, 0)
    field = torch.FloatTensor(field)
    field = torch.autograd.Variable(field, requires_grad=False)
    field = field.permute(0,2,3,1).cuda()
    
    I = Optimizer.get_identity_grid(None,field.size(2))
    expand = F.grid_sample(image, field + I)
    
    print("var field shape:", np.shape(field))
    print("var image shape:", np.shape(image))
    print("var I shape:", np.shape(I))
    print("var expand shape:", np.shape(expand))
    
    expand = (expand * 255.0).squeeze(0).squeeze(0).data.cpu().numpy().astype(np.uint8)
    
    if not already:
        already = True
        field_init = nn.AvgPool2d(kernel_size=16, stride=16)(field.permute(0,3,1,2)).detach().permute(0,2,3,1)
    
    stack.save('expand', 0, bbox, expand, is_field=False)
    stack.save('field', 0, bbox, field + I, is_field=True)
    
    field_plus_I = (field + I)
    
    field_plus_I = field_plus_I.permute(0,3,1,2)
    field_plus_I = nn.AvgPool2d(kernel_size=4, stride=4)(field_plus_I).permute(0,2,3,1)
    stack.save('field', 2, bbox, field_plus_I, is_field=True)
    
    field_plus_I = field_plus_I.permute(0,3,1,2)
    field_plus_I = nn.AvgPool2d(kernel_size=4, stride=4)(field_plus_I).permute(0,2,3,1)
    stack.save('field', 4, bbox, field_plus_I, is_field=True)
    
    field_plus_I = field_plus_I.permute(0,3,1,2)
    field_plus_I = nn.AvgPool2d(kernel_size=4, stride=4)(field_plus_I).permute(0,2,3,1)
    stack.save('field', 6, bbox, field_plus_I, is_field=True)
#     stack.save('expand', image_mip, bbox, np.abs(field1[1]*2550).astype(np.uint8), is_field=False)
    
#     widened = ndimage.filters.convolve(pixelperfect_mask, kernel, mode='constant', cval=0.0)
#     widened = (widened / (np.max(widened)+0.0) * 255).astype(np.uint8)
#     stack.save('widened', loss_mip, bbox, widened, is_field=False)

Downloading:   0%|          | 0/4 [00:00<?, ?it/s]
/usr/local/lib/python3.5/dist-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/usr/local/lib/python3.5/dist-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "
Downloading:   0%|          | 0/4 [00:00<?, ?it/s]


[[[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]] 0.036000000000000004
field shape: (2, 2048, 2048)
image shape: (512, 512)
var field shape: torch.Size([1, 2048, 2048, 2])
var image shape: torch.Size([1, 1, 512, 512])
var I shape: torch.Size([1, 2048, 2048, 2])
var expand shape: torch.Size([1, 1, 2048, 2048])


Downloading:   0%|          | 0/4 [00:00<?, ?it/s]


[[[-0.01706796 -0.01717023 -0.01737485 ...  0.          0.
    0.        ]
  [-0.01683574 -0.01693784 -0.01714213 ...  0.          0.
    0.        ]
  [-0.01660234 -0.01670426 -0.0169082  ...  0.          0.
    0.        ]
  ...
  [ 0.          0.          0.         ... -0.00038813 -0.00038677
   -0.00038609]
  [ 0.          0.          0.         ... -0.00038091 -0.00037959
   -0.00037891]
  [ 0.          0.          0.         ... -0.00037381 -0.00037251
   -0.00037185]]

 [[ 0.00100643  0.00123865  0.00147121 ...  0.          0.
    0.        ]
  [ 0.00089247  0.00112528  0.00135844 ...  0.          0.
    0.        ]
  [ 0.00066341  0.00089739  0.00113173 ...  0.          0.
    0.        ]
  ...
  [ 0.          0.          0.         ... -0.00071065 -0.0007034
   -0.00069617]
  [ 0.          0.          0.         ... -0.00070016 -0.00069302
   -0.0006859 ]
  [ 0.          0.          0.         ... -0.00069494 -0.00068785
   -0.00068079]]] 0.036000000000000004
field shape: (2,

Downloading:   0%|          | 0/2 [00:00<?, ?it/s]
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:40: RuntimeWarning: invalid value encountered in true_divide
Downloading:   0%|          | 0/2 [00:00<?, ?it/s]
/usr/local/lib/python3.5/dist-packages/numpy/core/_methods.py:26: RuntimeWarning: invalid value encountered in reduce
  return umr_maximum(a, axis, None, out, keepdims)


[[[nan nan nan ... nan nan nan]
  [nan nan nan ... nan nan nan]
  [nan nan nan ... nan nan nan]
  ...
  [nan nan nan ... nan nan nan]
  [nan nan nan ... nan nan nan]
  [nan nan nan ... nan nan nan]]

 [[nan nan nan ... nan nan nan]
  [nan nan nan ... nan nan nan]
  [nan nan nan ... nan nan nan]
  ...
  [nan nan nan ... nan nan nan]
  [nan nan nan ... nan nan nan]
  [nan nan nan ... nan nan nan]]] nan
field shape: (2, 2048, 2048)
image shape: (512, 512)
var field shape: torch.Size([1, 2048, 2048, 2])
var image shape: torch.Size([1, 1, 512, 512])
var I shape: torch.Size([1, 2048, 2048, 2])
var expand shape: torch.Size([1, 1, 2048, 2048])


In [53]:
nn.ZeroPad2d

torch.nn.modules.padding.ZeroPad2d

In [23]:
np.gradient([[1,2,3,4,5,4,3,2,1],[3,4,5,6,7,6,5,4,3],[1,2,3,4,5,4,3,2,1]])

[array([[ 2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [-2., -2., -2., -2., -2., -2., -2., -2., -2.]]),
 array([[ 1.,  1.,  1.,  1.,  0., -1., -1., -1., -1.],
        [ 1.,  1.,  1.,  1.,  0., -1., -1., -1., -1.],
        [ 1.,  1.,  1.,  1.,  0., -1., -1., -1., -1.]])]

In [72]:
x = 1015
y = -1005
while np.max(widened[y+x:y-x,y+x:y-x]) < np.max(widened)-21:
    print(y)
    y += 10
print("y:", y)
widened[y+x:y-x,y+x:y-x]
# np.max(widened)

-1005
-995
-985
-975
-965
-955
-945
-935
-925
-915
-905
-895
-885
-875
-865
-855
-845
-835
-825
-815
-805
-795
-785
-775
-765
-755
-745
-735
-725
-715
-705
-695
-685
-675
-665
-655
-645
-635
-625
-615
-605
-595
-585
-575
-565
-555
-545
-535
-525
-515
-505
-495
-485
-475
-465
-455
-445
-435
-425
-415
-405
-395
-385
-375
-365
-355
-345
-335
-325
-315
-305
-295
-285
-275
-265
-255
-245
-235
-225
-215
-205
-195
-185
y: -175


array([[232, 232, 232, 232, 232, 233, 233, 233, 233, 233, 233, 233, 233,
        233, 233, 233, 233, 233],
       [232, 232, 232, 232, 232, 233, 233, 233, 233, 233, 233, 233, 233,
        233, 233, 233, 233, 233],
       [232, 232, 232, 232, 232, 233, 233, 233, 233, 233, 233, 233, 233,
        233, 233, 233, 233, 233],
       [232, 232, 232, 232, 232, 233, 233, 233, 233, 233, 233, 233, 233,
        233, 233, 233, 233, 233],
       [232, 232, 232, 232, 232, 233, 233, 233, 233, 233, 233, 233, 233,
        233, 233, 233, 233, 233],
       [232, 232, 232, 232, 232, 233, 233, 233, 233, 233, 233, 233, 233,
        233, 233, 233, 233, 233],
       [232, 232, 232, 232, 232, 233, 233, 233, 233, 233, 233, 233, 233,
        233, 233, 233, 233, 233],
       [232, 232, 232, 232, 232, 233, 233, 233, 233, 233, 233, 233, 233,
        233, 233, 233, 233, 233],
       [232, 232, 232, 232, 232, 233, 233, 233, 233, 233, 233, 233, 233,
        233, 233, 233, 233, 233],
       [232, 232, 232, 232, 232, 233,

In [77]:
x = 1018
y = -160
widened[y+x:y-x,y+x:y-x]
# np.max(widened)

array([[234, 234, 234, 233, 233, 233, 233, 233, 233, 233, 233, 232],
       [234, 234, 234, 233, 233, 233, 233, 233, 233, 233, 233, 232],
       [234, 234, 234, 234, 233, 233, 233, 233, 233, 233, 233, 232],
       [234, 234, 234, 234, 233, 233, 233, 233, 233, 233, 233, 232],
       [234, 234, 234, 234, 234, 233, 233, 233, 233, 233, 233, 233],
       [234, 234, 234, 234, 234, 233, 233, 233, 233, 233, 233, 233],
       [234, 234, 234, 234, 234, 233, 233, 233, 233, 233, 233, 233],
       [234, 234, 234, 234, 234, 234, 233, 233, 233, 233, 233, 233],
       [234, 234, 234, 234, 234, 234, 233, 233, 233, 233, 233, 233],
       [234, 234, 234, 234, 234, 234, 233, 233, 233, 233, 233, 233],
       [234, 234, 234, 234, 234, 234, 234, 233, 233, 233, 233, 233],
       [234, 234, 234, 234, 234, 234, 234, 233, 233, 233, 233, 233]],
      dtype=uint8)

In [140]:
Variable(field_init.data.cpu()).cuda().detach()

Variable containing:
(0 ,0 ,.,.) = 
1.00000e-02 *
  0.0000  0.0000
  0.0000  0.0000
  0.0000  0.0000
       ⋮        
  0.0000  0.0000
  0.0000  0.0000
  0.0000  0.0000

(0 ,1 ,.,.) = 
1.00000e-02 *
  0.0000  0.0000
  0.0000  0.0000
  0.0000  0.0000
       ⋮        
  0.0000  0.0000
  0.0000  0.0000
  0.0000  0.0000

(0 ,2 ,.,.) = 
1.00000e-02 *
  0.0000  0.0000
  0.0000  0.0000
  0.0000  0.0000
       ⋮        
  0.0000  0.0000
  0.0000  0.0000
  0.0000  0.0000
   ...

(0 ,29,.,.) = 
1.00000e-02 *
  0.0000  0.0000
  0.0000  0.0000
  0.0000  0.0000
       ⋮        
  0.0000  0.0000
  0.0000  0.0000
  0.0000  0.0000

(0 ,30,.,.) = 
1.00000e-02 *
  0.0000  0.0000
  0.0000  0.0000
  0.0000  0.0000
       ⋮        
  0.0000  0.0000
  0.0000  0.0000
  0.0000  0.0000

(0 ,31,.,.) = 
1.00000e-02 *
  0.0000  0.0000
  0.0000  0.0000
  0.0000  0.0000
       ⋮        
  0.0000  0.0000
  0.0000  0.0000
  0.0000  0.0000
[torch.cuda.FloatTensor of size 1x32x32x2 (GPU 0)]